In [1]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [2]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/parse-faq.ipynb

In [10]:
import minsearch
import json
from dotenv import load_dotenv
from groq import Groq
import os

load_dotenv()

True

In [ ]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [8]:
documents[0]

{'text': "dThe purpose of this document is to capture frequently asked technical questions\nThe next cohort starts in Jan 2025. More info at DTC Article.\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start？',
 'course': 'data-engineering-zoomcamp'}

In [9]:
Index = minsearch.Index(
    text_fields = ['question','section','text'],
    keyword_fields = ['course']
)

Index.fit(documents)

In [ ]:
ques = "how to parse document files with python?"

In [ ]:
boost = {'question': 3, 'section' : 0.4}

results = Index.search(
    query=ques,
    filter_dict={'course':'mlops-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [ ]:
results

#### from search script TextSearch Class function

In [ ]:
from Search_uncln import TextSearch

index = TextSearch(
    text_fields = ['question','section','text'],
)
index.fit(documents)

se_results =index.search(
    query = ques,
    n_results= 2, 
    filters= {'course':'mlops-zoomcamp'},
boost = {'question':6, 'section':0.2},
)

In [ ]:
se_results

### chatcompletition, Prompt Template, LLM api call

In [ ]:
import minsearch
import json
from dotenv import load_dotenv
from groq import Groq
import os

load_dotenv()

In [21]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

Index = minsearch.Index(
    text_fields = ['question','section','text'],
    keyword_fields = ['course']
)

Index.fit(documents)

In [ ]:
prompt_template = """ 
You are an expert machine learning and mlops engineering helping a junior engineer as an assitant and guide. 
Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering.
if the CONTEXT does not contain the answer, Output "Not FOUND in the context given" and explain your answer with reasons.

QUESTION: {question}

CONTEXT: {context}
""".strip()

context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

prompt = prompt_template.format(question=ques, context=context).strip()

In [ ]:
print(prompt)

In [ ]:

client = Groq(
    api_key=os.getenv("GROQ_API_KEY"),
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
)
response.choices[0].message.content

In [11]:
def search(query):
    boost = {'question': 3, 'section' : 0.4}

    results = Index.search(
        query=query,
        filter_dict={'course':'mlops-zoomcamp'},
        boost_dict=boost,
        num_results=5
)
    return results

In [24]:
def build_prompt(query, search_results):
    prompt_template = """ 
You are an expert machine learning and mlops engineering helping a junior engineer as an assitant and guide. 
Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering. DO NOT USE OTHER CONTENT OTHER THAN GIVEN CONTEXT!
if the CONTEXT does not contain the answer, Output "Not FOUND in the context given" and explain your answer with reasons.

QUESTION: {question}

CONTEXT: {context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [25]:
def llm_call(prompt):
    client = Groq(
    api_key=os.getenv("GROQ_API_KEY"),
)
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
)
    return print(response.choices[0].message.content)

In [30]:
Query = "How to use mlflow for experiment tracking?"
def rag(Query):
    
    search_results = search(Query)
    Prompt = build_prompt(Query, search_results)
    answer = llm_call(Prompt)
    return answer

In [31]:
rag(Query)

Based on the given context, I will help you with your question on how to use MLflow for experiment tracking.

To use MLflow for experiment tracking, you can follow the guidelines provided in the "Module 2: Experiment tracking" section of the context.

If you are experiencing issues with your experiment not being visible in the MLflow UI, you can refer to the solution provided for the question "Experiment not visible in MLflow UI" which suggests checking the tracking URI and ensuring that it is correct. The tracking URI can be an absolute or relative path to the mlflow.db file.

If you want to delete an experiment permanently from MLflow UI, you can refer to the solution provided for the question "How to Delete an Experiment Permanently from MLFlow UI" which suggests using a SQL script to delete the experiment.

Please let me know if you have any further questions or if there's anything else I can help you with.


In [32]:
rag("How to use Kafka?")

I see you're asking about how to use Kafka!

The context doesn't contain any information about Kafka. I searched through the provided text, and Kafka is not mentioned at all. Therefore, I must output: "Not FOUND in the context given"

The reason is that Kafka is not mentioned in the provided text, neither as a topic, nor a technology, nor a keyword. The text only discusses Git,Experiment tracking with MLflow, and general course questions.
